In [2]:
import matplotlib.pyplot as plt
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math

In [3]:
def Geometric_Slerp(P0,P1,r,cant_puntos):

    omega = math.acos((P0[0]*P1[0]+P0[1]*P1[1]+P0[2]*P1[2])/(r*r)) #La fórmula falla cuando los puntos son los extremos de un diametro

    x_geo = [P0[0]*np.sin(omega*(1-t))/np.sin(omega)+P1[0]*np.sin(omega*t)/np.sin(omega) for t in np.linspace(0.0, 1.0, num=cant_puntos)]

    y_geo = [P0[1]*np.sin(omega*(1-t))/np.sin(omega)+P1[1]*np.sin(omega*t)/np.sin(omega) for t in np.linspace(0.0, 1.0, num=cant_puntos)]

    z_geo = [P0[2]*np.sin(omega*(1-t))/np.sin(omega)+P1[2]*np.sin(omega*t)/np.sin(omega) for t in np.linspace(0.0, 1.0, num=cant_puntos)]

    return x_geo,y_geo,z_geo

In [4]:
def cross(a, b):
    c = [a[1]*b[2] - a[2]*b[1],
         a[2]*b[0] - a[0]*b[2],
         a[0]*b[1] - a[1]*b[0]]

    return c

In [5]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [6]:
from scipy.optimize import fsolve
import math

def equations(p):
    x, y, z = p

    return (In_rot[0]*x + In_rot[1]*y + In_rot[2]*z - np.cos(theta_cap),\
            normal_vector[0]*x + normal_vector[1]*y + normal_vector[2]*z,\
           x*x+y*y+z*z-1)

In [7]:

def color_triangle(x, y, z, V0, V1, V2):
    
    Centroid = [V0[0]+V1[0]+V2[0],V0[1]+V1[1]+V2[1],V0[2]+V1[2]+V2[2]]
   
    x_cons_0 = V0[1]*V1[2] - V0[2]*V1[1]
    y_cons_0 = V0[2]*V1[0] - V0[0]*V1[2]
    z_cons_0 = V0[0]*V1[1] - V0[1]*V1[0]
    
    if Centroid[0]*x_cons_0+Centroid[1]*y_cons_0+Centroid[2]*z_cons_0<0:
        x_cons_0 = -x_cons_0
        y_cons_0 = -y_cons_0
        z_cons_0 = -z_cons_0
    
    x_cons_1 = V2[1]*V1[2] - V2[2]*V1[1]
    y_cons_1 = V2[2]*V1[0] - V2[0]*V1[2]
    z_cons_1 = V2[0]*V1[1] - V2[1]*V1[0]
    
    if Centroid[0]*x_cons_1+Centroid[1]*y_cons_1+Centroid[2]*z_cons_1<0:
        x_cons_1 = -x_cons_1
        y_cons_1 = -y_cons_1
        z_cons_1 = -z_cons_1
    
    x_cons_2 = V0[1]*V2[2] - V0[2]*V2[1]
    y_cons_2 = V0[2]*V2[0] - V0[0]*V2[2]
    z_cons_2 = V0[0]*V2[1] - V0[1]*V2[0]
    
    if Centroid[0]*x_cons_2+Centroid[1]*y_cons_2+Centroid[2]*z_cons_2<0:
        x_cons_2 = -x_cons_2
        y_cons_2 = -y_cons_2
        z_cons_2 = -z_cons_2
    
    total = np.sign(x_cons_1*x + y_cons_1*y + z_cons_1*z) + np.sign(x_cons_2*x + y_cons_2*y + z_cons_2*z) + np.sign(x_cons_0*x + y_cons_0*y + z_cons_0*z)
   
    total[total < 3] = 0
    
    return total/3


In [8]:
def spherical_area(a, b, c):
        t = abs( inner(a, cross(b, c) ) )
        t /= 1 + inner(a,b) + inner(b,c) + inner(a,c)
        return 2*arctan(t)

In [10]:
V0_iso = [1, 0, 0]
V1_iso = [0.3406884710289558, 0.9328944522580684, 0.11678744219579273]
V2_iso = [0.3406884710289558, 0.12410827817548943, 0.9319487652208257]

spherical_area(V0_iso, V1_iso, V2_iso)

0.8000000000014511

In [11]:
from scipy.optimize import fsolve
import random

V0_aux = [1, 0, 0]
aux_b = -0.2
V1_aux = [aux_b, sqrt(1-aux_b*aux_b), 0]

target_area = spherical_area(V0_iso, V1_iso, V2_iso)

def third_vertex_same_area(p):
    
    c1, c2, c3 = p

    mod_V3_aux = sqrt(c1*c1+c2*c2+c3*c3)
    V3_aux = [c1/mod_V3_aux, c2/mod_V3_aux, c3/mod_V3_aux]
    
    output = (target_area - spherical_area(V0_aux, V1_aux, V3_aux))
    
    return mod_V3_aux-1, output, output
    
# Sanity check
c1, c2, c3 = fsolve(third_vertex_same_area, [0,0,1])
print(c1, c2, c3, sqrt(c1*c1+c2*c2+c3*c3))
print(target_area - spherical_area(V0_aux, V1_aux, [c1, c2, c3]))

0.624241438623307 0.33879841108016295 0.703944786860394 0.9999999999964323
-1.3470224935474562e-11


In [12]:
from scipy.optimize import fsolve

def vertices_iso(p):
    b1, b2, b3, c1, c2, c3 = p
    
    T1 = 0.5*(b1+c1)
    T2 = 0.5*(b2+c2)
    T3 = 0.5*(b3+c3)
    
    mod_T = sqrt(T1*T1+T2*T2+T3*T3)
    T = [T1/mod_T, T2/mod_T, T3/mod_T]
    
    return 1-b1*b1-b2*b2-b3*b3, 1-c1*c1-c2*c2-c3*c3, (c1*V2[0] + c2*V2[1] + c3*V2[2]) - (b1*V2[0] + b2*V2[1] + b3*V2[2]), (c1*V2[0] + c2*V2[1] + c3*V2[2])-(b1*c1 + b2*c2 + b3*c3), area_iso - spherical_area([b1, b2, b3], [c1, c2, c3], V2), spherical_area(V2, T, [c1,c2,c3])-spherical_area(V2, [b1,b2,b3], T)

# Vertices
area_iso = 0.8
V2 = [1, 0, 0]

b1, b2, b3, c1, c2, c3 = fsolve(vertices_iso, [0,1,0, 0,0,1])

T1 = 0.5*(b1+c1)
T2 = 0.5*(b2+c2)
T3 = 0.5*(b3+c3)
    
mod_T = sqrt(T1*T1+T2*T2+T3*T3)
T = [T1/mod_T, T2/mod_T, T3/mod_T]

print(b1, b2, b3, c1, c2, c3)
print(vertices_iso([b1, b2, b3, c1, c2, c3]))
spherical_area([b1, b2, b3], [c1, c2, c3], V2)
print(spherical_area(V2, T, [c1,c2,c3]),spherical_area(V2, [b1,b2,b3], T))

0.3406884710289558 0.9328944522580684 0.11678744219579273 0.3406884710289558 0.12410827817548943 0.9319487652208257
(-5.647583095624853e-13, -3.539391002504999e-13, 0.0, 2.531308496145357e-14, -1.4510614931850796e-12, 1.9317880628477724e-14)
0.4000000000006752 0.4000000000006559


In [13]:
# Sanity check
spherical_area(V2, T, [c1,c2,c3])-spherical_area(V2, [b1,b2,b3], T)

1.9317880628477724e-14

In [14]:
from scipy.optimize import fsolve

def vertices(p):
    b1, b2, b3, c1, c2, c3 = p
    
    T1 = 0.5*(b1+c1)
    T2 = 0.5*(b2+c2)
    T3 = 0.5*(b3+c3)
    
    mod_T = sqrt(T1*T1+T2*T2+T3*T3)
    T = [T1/mod_T, T2/mod_T, T3/mod_T]
    
    return 1-b1*b1-b2*b2-b3*b3, 1-c1*c1-c2*c2-c3*c3, cos(dist_aT)-(T1*V0[0] - T2*V0[1] - T3*V0[2])/mod_T, spherical_area(V0, T, [c1,c2,c3])-0.5*spherical_area(V0, V1, V2), spherical_area(V0, [b1,b2,b3], T)-spherical_area(V0, T, [c1,c2,c3]), spherical_area(V0, V1, V2)-spherical_area(V0, [b1,b2,b3], T)-spherical_area(V0, T, [c1,c2,c3])

# Vertices
V0 = [1, 0, 0]
V1 = [0.3406884710289558, 0.9328944522580684, 0.11678744219579273]
V2 = [0.3406884710289558, 0.12410827817548943, 0.9319487652208257]

dist_aT = .22 * pi

b1, b2, b3, c1, c2, c3 = fsolve(vertices, [0,1,0, 0,0,1])

# Sanity check
print(b1, b2, b3, c1, c2, c3)
print(vertices([b1, b2, b3, c1, c2, c3]))
print(spherical_area(V0, V1, V2), spherical_area([b1, b2, b3], [c1, c2, c3], V0))

0.3995691276168492 0.7476891904854146 -0.5303823023903146 0.39956912762129915 -0.19648768029759267 0.8953977349474517
(-2.07556194453673e-12, -5.802025526691068e-12, 3.527289571536585e-12, -5.199563002378227e-12, 1.1649570197391768e-12, 9.234168985017277e-12)
0.8000000000014511 0.7999999999941646


In [15]:
from scipy.optimize import fsolve

def prop_sides(p):
    a1, a2, b1, b2, b3, c1, c3 = p
    
    a3 = a2
    c2 = c1*In[1]/In[0]
    
    # print(a1, a2, a3, b1, b2, b3, c1, c2, c3)

    
    cos_M0M1 = a1*b1+a2*b2+a3*b3
    cos_M0M2 = a1*c1+a2*c2+a3*c3
    cos_M1M2 = c1*b1+c2*b2+c3*b3
    
    
    cos_M0In = a1*In[0]+a2*In[1]+a3*In[2]
    cos_M1In = b1*In[0]+b2*In[1]+b3*In[2]
    cos_M2In = c1*In[0]+c2*In[1]+c3*In[2]
    
    return 1-a1*a1-a2*a2-a3*a3, 1-b1*b1-b2*b2-b3*b3, 1-c1*c1-c2*c2-c3*c3, cos_prop_L_V0V1-cos_M0M1, cos_prop_L_V0V2-cos_M0M2,cos_prop_L_V1V2-cos_M1M2, cos_prop_L_V0In-cos_M0In

V0 = [1,0,0]
V1 = [1/sqrt(2),1/sqrt(2),0]
V2 = [0,0,1]

L_V0V1 = math.acos(V0[0]*V1[0]+V0[1]*V1[1]+V0[2]*V1[2])
L_V0V2 = math.acos(V0[0]*V2[0]+V0[1]*V2[1]+V0[2]*V2[2])
L_V1V2 = math.acos(V2[0]*V1[0]+V2[1]*V1[1]+V2[2]*V1[2])

# Incenter
In = [0.8628562, 0.3574067, 0.3574067]
prop = 0.6

cos_prop_L_V0V1 = cos(prop*L_V0V1)
cos_prop_L_V0V2 = cos(prop*L_V0V2)
cos_prop_L_V1V2 = cos(prop*L_V1V2)

L_V0In = math.acos(V0[0]*In[0]+V0[1]*In[1]+V0[2]*In[2])
L_V1In = math.acos(V1[0]*In[0]+V1[1]*In[1]+V1[2]*In[2])
L_V2In = math.acos(V2[0]*In[0]+V2[1]*In[1]+V2[2]*In[2])



cos_prop_L_V0In = cos(prop*L_V0In)
cos_prop_L_V1In = cos(prop*L_V1In)
cos_prop_L_V2In = cos(prop*L_V2In)


print(cos_prop_L_V0V1,cos_prop_L_V0V2,cos_prop_L_V1V2)
print(cos_prop_L_V0In,cos_prop_L_V1In,cos_prop_L_V2In)
print("-----------------------")

a1, a2, b1, b2, b3, c1, c3 = fsolve(prop_sides, [1,0,1/sqrt(3),1/sqrt(3),1/sqrt(3),1/sqrt(6),2/sqrt(6)])
a3 = a2
c2 = c1*In[1]/In[0]
print(a1, a2, a3, b1, b2, b3, c1, c2, c3)
print(prop_sides([a1, a2, b1, b2, b3, c1, c3]))
print(spherical_area(V0, V1, V2), spherical_area([b1, b2, b3], [c1, c2, c3], [a1, a2, a3]))

0.8910065241883679 0.5877852522924731 0.5877852522924731
0.949880883876946 0.9498808732713284 0.7497024602519698
-----------------------
0.9776202076232937 0.1487594195686591 0.1487594195686591 0.8005372044576908 0.5807636988654586 0.1478300028923951 0.4397005908238513 0.1821299275063481 0.8794839850427625
(-1.7422065179406587e-10, 6.040106748095297e-10, -1.100974866830029e-11, -1.860989140567426e-11, -3.764422107366272e-11, 9.058087613311727e-11, 1.1102230246251565e-16)
0.7853981633974483 0.23478470859241837


In [16]:
from scipy.optimize import fsolve

def prop_sides(p):
    a1, a2, b1, b2, b3, c1, c3 = p
    
    a3 = a2
    c2 = c1*In[1]/In[0]
    
    # print(a1, a2, a3, b1, b2, b3, c1, c2, c3)

    
    cos_M0M1 = a1*b1+a2*b2+a3*b3
    cos_M0M2 = a1*c1+a2*c2+a3*c3
    cos_M1M2 = c1*b1+c2*b2+c3*b3
    
    
    cos_M0In = a1*In[0]+a2*In[1]+a3*In[2]
    cos_M1In = b1*In[0]+b2*In[1]+b3*In[2]
    cos_M2In = c1*In[0]+c2*In[1]+c3*In[2]
    
    return 1-a1*a1-a2*a2-a3*a3, 1-b1*b1-b2*b2-b3*b3, 1-c1*c1-c2*c2-c3*c3, cos_prop_L_V0V1-cos_M0M1, cos_prop_L_V0V2-cos_M0M2,cos_prop_L_V1V2-cos_M1M2, cos_prop_L_V0In-cos_M0In

V0 = [1,0,0]
V1 = [1/sqrt(2),1/sqrt(2),0]
V2 = [0,0,1]

L_V0V1 = math.acos(V0[0]*V1[0]+V0[1]*V1[1]+V0[2]*V1[2])
L_V0V2 = math.acos(V0[0]*V2[0]+V0[1]*V2[1]+V0[2]*V2[2])
L_V1V2 = math.acos(V2[0]*V1[0]+V2[1]*V1[1]+V2[2]*V1[2])

L_V0In = math.acos(V0[0]*In[0]+V0[1]*In[1]+V0[2]*In[2])
L_V1In = math.acos(V1[0]*In[0]+V1[1]*In[1]+V1[2]*In[2])
L_V2In = math.acos(V2[0]*In[0]+V2[1]*In[1]+V2[2]*In[2])

# Incenter
In = [0.8628562, 0.3574067, 0.3574067]

a1 = []
a2 = []
a3 = []
b1 = []
b2 = []
b3 = []
c1 = []
c2 = []
c3 = []

for prop in np.linspace(0.2,1.1,21):
    
    cos_prop_L_V0V1 = cos(prop*L_V0V1)
    cos_prop_L_V0V2 = cos(prop*L_V0V2)
    cos_prop_L_V1V2 = cos(prop*L_V1V2)
    
    cos_prop_L_V0In = cos(prop*L_V0In)
    cos_prop_L_V1In = cos(prop*L_V1In)
    cos_prop_L_V2In = cos(prop*L_V2In)
    
    a1_aux, a2_aux, b1_aux, b2_aux, b3_aux, c1_aux, c3_aux = fsolve(prop_sides, [1,0,1/sqrt(3),1/sqrt(3),1/sqrt(3),1/sqrt(6),2/sqrt(6)])
    a3_aux = a2_aux
    c2_aux = c1_aux*In[1]/In[0]
    print(a1_aux, a2_aux, a3_aux, b1_aux, b2_aux, b3_aux, c1_aux, c2_aux, c3_aux)
    
    a1.append(a1_aux)
    a2.append(a2_aux)
    a3.append(a3_aux)
    b1.append(b1_aux)
    b2.append(b2_aux)
    b3.append(b3_aux)
    c1.append(c1_aux)
    c2.append(c2_aux)
    c3.append(c3_aux)

import pandas as pd

data = {'a1': a1, 'a2': a2, 'a3': a3, 'b1': b1, 'b2': b2, 'b3': b3, 'c1': c1, 'c2': c2, 'c3': c3, 'prop' : np.linspace(0.2,1.1,21)}

df = pd.DataFrame(data)

print (df)

df.to_csv('../Results/data_figS8b.csv', index=False)

0.9114824571273529 0.29086055947408684 0.29086055947408684 0.8518802192712722 0.43601700049727754 0.2901538684118318 0.762411232021056 0.3158010366960103 0.564799804073887
0.9210310620683492 0.27541040556673085 0.27541040556673085 0.8480900257225692 0.45314971393979603 0.2745881369181128 0.7337372725203782 0.30392389512703166 0.6076675741279205
0.9300559679543221 0.25980367235674906 0.25980367235674906 0.8438061753509616 0.4700733993445009 0.25888634168658325 0.7029625295806691 0.291176580664402 0.648891270345054
0.9385520505961641 0.24404922487191524 0.24404922487191524 0.8390245743235929 0.4867854805919004 0.24305896314862613 0.6701546691205056 0.27758712144613645 0.6883589975170236
0.9465144803997637 0.2281560194760745 0.2281560194760745 0.8337410267004589 0.5032833309191954 0.22711624602336997 0.6353847772702447 0.26318496230819594 0.7259613353476809
0.9539387314015173 0.21213309118461315 0.21213309118461315 0.8279512210064877 0.5195642545839987 0.21106814301752969 0.59872714484016